In [ ]:
import syft as sy
import recordlinkage
from syft.abstract_node import NodeType
from recordlinkage.datasets import load_febrl4

# Create Nodes

## Low side

create enclave node

In [ ]:
embassador_node_low = sy.Orchestra.launch(name="ambassador node",local_db=True, reset=True)

Create canada node & italy node

In [ ]:
ca_node_low = sy.Orchestra.launch(name="canada low", local_db=True, reset=True)
it_node_low = sy.Orchestra.launch(name="italy low", local_db=True, reset=True)

In [ ]:
gateway_node_low = sy.orchestra.launch(
    name="gateway low",
    node_type=sy.NodeType.GATEWAY,
    local_db=True,
    reset=True,
    dev_mode=True,
)

## High side

In [ ]:
enclave_node_high = sy.orchestra.launch(name="enclave node", node_type=sy.NodeType.ENCLAVE, reset=True)
ca_node_high = sy.Orchestra.launch(name="canada high", local_db=True, reset=True)
it_node_high = sy.Orchestra.launch(name="italy high", local_db=True, reset=True)

In [ ]:
gateway_node_high = sy.orchestra.launch(name="gateway high", node_type=sy.NodeType.GATEWAY, local_db=True, reset=True, dev_mode=True)

# DOs

## Login

### Low side

In [ ]:
do_ca_client_low = ca_node_low.login(email="info@openmined.org", password="changethis")
do_it_client_low = it_node_low.login(email="info@openmined.org", password="changethis")
embassador_client_low = embassador_node_low.login(email="info@openmined.org", password="changethis")

### High side

In [ ]:
do_ca_client_high = ca_node_high.login(email="info@openmined.org", password="changethis")
do_it_client_high = it_node_high.login(email="info@openmined.org", password="changethis")

## Connect to network

In [ ]:
# TODO: add security layer here

In [ ]:
enclave_client_high = enclave_node_high.client

In [ ]:
# gateway_root_client.register(name="", email="info@openmined.org", password="changethis")
# gateway_root_client.register(name="", email="info@openmined.org", password="changethis")

In [ ]:
res = do_ca_client_low.connect_to_gateway(handle=gateway_node_low)  # add credentials here

In [ ]:
res = do_ca_client_low.connect_to_gateway(handle=gateway_node_low)  # add credentials here
res = do_it_client_low.connect_to_gateway(handle=gateway_node_low)  # add credentials here

In [ ]:
res = do_ca_client_high.connect_to_gateway(handle=gateway_node_high)
res = do_it_client_high.connect_to_gateway(handle=gateway_node_high)

## Also for ambassador

In [ ]:
# TODO: who is going to be responsible for connecting the enclave to the gateway
res = enclave_client_high.connect_to_gateway(handle=gateway_node_high)

In [ ]:
res = embassador_client_low.connect_to_gateway(handle=gateway_node_low)  # add credentials here

## Upload dataset

### Low side

In [ ]:
# Using public datasets from  Freely Extensible Biomedical Record Linkage (Febrl)
canada_census_data_low, italy_census_data_low = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data_low, italy_census_data_low],
    [do_ca_client_low, do_it_client_low],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

### High side

In [ ]:
# Using public datasets from  Freely Extensible Biomedical Record Linkage (Febrl)
canada_census_data_high, italy_census_data_high = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data_high, italy_census_data_high],
    [do_ca_client_high, do_it_client_high],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

## create accounts for DS

### Low side

In [ ]:
for client in [do_ca_client_low, do_it_client_low]:
    client.register(name="Sheldon", email="sheldon@caltech.edu", password="changethis")

In [ ]:
embassador_client_low.register(name="Sheldon", email="sheldon@caltech.edu", password="changethis")

## Create account for embassador

### High Side

In [ ]:
for client in [do_ca_client_high, do_it_client_high]:
    client.register(name="Sheldon", email="sheldon@caltech.edu", password="changethis")

# DS Low Side

## DS Get proxy clients

### Low side

In [ ]:
ds_gateway_client_low = gateway_node_low.client

In [ ]:
ds_gateway_client_low.domains

In [ ]:
ds_ca_proxy_client_low = ds_gateway_client_low.proxy_client_for(name="canada low", email="sheldon@caltech.edu", password="changethis")
ds_it_proxy_client_low = ds_gateway_client_low.proxy_client_for(name="italy low", email="sheldon@caltech.edu", password="changethis")
ds_amb_proxy_client_low = ds_gateway_client_low.proxy_client_for(name="ambassador node", email="sheldon@caltech.edu", password="changethis")

## Login (Deprecated)

In [ ]:
# ds_client = embassador_node_low.login(name="Sheldon", email="sheldon@caltech.edu", password="changethis")
# ds_client_it_low = it_node_low.login(email="sheldon@caltech.edu" , password="changethis")
# ds_client_ca_low = ca_node_low.login(email="sheldon@caltech.edu" , password="changethis")

## Find datasets

In [ ]:
canada_census_data = ds_ca_proxy_client_low.datasets[-1].assets[0]
italy_census_data = ds_it_proxy_client_low.datasets[-1].assets[0]

## Create Request

In [ ]:
@sy.syft_function_single_use(canada_census_data=canada_census_data, italy_census_data=italy_census_data)
def compute_census_matches(canada_census_data, italy_census_data):
    import recordlinkage

    # Index step
    indexer = recordlinkage.Index()
    indexer.block("given_name")

    candidate_links = indexer.index(canada_census_data, italy_census_data)

    # Comparison step
    compare_cl = recordlinkage.Compare()

    compare_cl.exact("given_name", "given_name", label="given_name")
    compare_cl.string("surname", "surname", method="jarowinkler", threshold=0.85, label="surname")
    compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
    compare_cl.exact("suburb", "suburb", label="suburb")
    compare_cl.exact("state", "state", label="state")
    compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")

    features = compare_cl.compute(candidate_links, canada_census_data, italy_census_data)

    # Classification step
    matches = features[features.sum(axis=1) > 3]

    return len(matches)

In [ ]:
# Checking result of mock data execution
mock_result = compute_census_matches(
    canada_census_data=canada_census_data.mock,
    italy_census_data=italy_census_data.mock,
)
mock_result

In [ ]:
ds_amb_proxy_client_low.code.request_code_execution(compute_census_matches)

# Ambassador flow

## Check Code Low Side

In [ ]:
embassador_client_low.requests[0].code

## Login to High Side

In [ ]:
amb_gateway_client_high = gateway_node_high.client

In [ ]:
amb_gateway_client_high.domains

In [ ]:
amb_ca_proxy_client_high = amb_gateway_client_high.proxy_client_for(name="canada high", email="sheldon@caltech.edu", password="changethis")
amb_it_proxy_client_high = amb_gateway_client_high.proxy_client_for(name="italy high", email="sheldon@caltech.edu", password="changethis")
amb_enclave_proxy_client_high = amb_gateway_client_high.proxy_client_for(name="enclave node")

In [ ]:
# TODO: FIX
amb_enclave_proxy_client_high.login(name="Sheldon", email="sheldon@caltech.edu", password="changethis", register=True)

## Login High Side (DEPRECATED)

In [ ]:
# amb_it_proxy_client_high = it_node_high.login(email="sheldon@caltech.edu" , password="changethis")
# amb_ca_proxy_client_high = ca_node_high.login(email="sheldon@caltech.edu" , password="changethis")

In [ ]:
# # this also creates a guest client
# embassador_client_high = enclave_node_high.login(email="info@openmined.org", password="changethis",
#                                                  name="Signor Ambassador", register=True)

## Find Datasets High side

In [ ]:
canada_census_data_high = amb_ca_proxy_client_high.datasets[-1].assets[0]
italy_census_data_high = amb_it_proxy_client_high.datasets[-1].assets[0]

Copy code from the request

## Submit code High side

In [ ]:
@sy.syft_function_single_use(canada_census_data=canada_census_data_high, italy_census_data=italy_census_data_high)
def compute_census_matches_high(canada_census_data, italy_census_data):
    import recordlinkage

    # Index step
    indexer = recordlinkage.Index()
    indexer.block("given_name")

    candidate_links = indexer.index(canada_census_data, italy_census_data)

    # Comparison step
    compare_cl = recordlinkage.Compare()

    compare_cl.exact("given_name", "given_name", label="given_name")
    compare_cl.string(
        "surname", "surname", method="jarowinkler", threshold=0.85, label="surname"
    )
    compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
    compare_cl.exact("suburb", "suburb", label="suburb")
    compare_cl.exact("state", "state", label="state")
    compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")

    features = compare_cl.compute(
        candidate_links, canada_census_data, italy_census_data
    )

    # Classification step
    matches = features[features.sum(axis=1) > 3]

    return len(matches)

In [ ]:
# Checking result of mock data execution
mock_result = compute_census_matches_high(
    canada_census_data=canada_census_data_high.mock,
    italy_census_data=italy_census_data_high.mock,
)
mock_result

In [ ]:
# note that this is not embassador_client_high.**code**.request_code_execution
amb_enclave_proxy_client_high.request_code_execution(compute_census_matches_high)

## DOs Approve High Side

In [ ]:
do_ca_client_high.requests[0].approve()

In [ ]:
do_it_client_high.requests[0].approve()

## Embassdor gets result from High Side

In [ ]:
amb_enclave_proxy_client_high.code.get_all()[-1].status

In [ ]:
result_pointer = amb_enclave_proxy_client_high.code.compute_census_matches_high(
    canada_census_data=canada_census_data_high,
    italy_census_data=italy_census_data_high,
)

result_pointer

In [ ]:
real_result = result_pointer.get()
real_result

## Ambassador Deposits Result

In [ ]:
embassador_client_low.requests[0].accept_by_depositing_result(real_result)

# DS

##  Get result from low side

In [ ]:
ds_amb_proxy_client_low.code.get_all()[-1].status

In [ ]:
result_pointer = ds_amb_proxy_client_low.code.compute_census_matches(
    canada_census_data=canada_census_data,
    italy_census_data=italy_census_data,
)

In [ ]:
result_pointer

In [ ]:
real_result = result_pointer.get()
real_result